In [6]:
import numpy  as np
import math
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from numpy import *

###读入数据 & 设置截距项 & 五折交叉验证训练集测试集划分
boston = load_boston()
y = boston['target']
y = transpose(mat(ravel(y)))
x = boston['data']
x0 = mat(zeros((506, 1))) + 1
x = column_stack((x, x0)) 
names = boston['feature_names']
x_train1=x[101:506]
x_test1=x[0:101]
y_train1=y[101:506]
y_test1=y[0:101]

x_train2=np.vstack((x[1:101],x[202:507]))
x_test2=x[101:202]
y_train2=np.vstack((y[1:101],y[202:507]))
y_test2=y[101:202]

x_train3=np.vstack((x[1:202],x[304:507]))
x_test3=x[202:304]
y_train3=np.vstack((y[1:202],y[304:507]))
y_test3=y[202:304]

x_train4=np.vstack((x[1:304],x[405:507]))
x_test4=x[304:405]
y_train4=np.vstack((y[1:304],y[405:507]))
y_test4=y[304:405]

x_train5=x[1:405]
x_test5=x[405:507]
y_train5=y[1:405]
y_test5=y[405:507]


In [9]:
##绝对收敛标准，lamda网格搜索，五折交叉验证
eps = 1e-9
def shrink(z, t):
    assert(t>0)
    x0 = np.zeros(z.shape)
    x0[z>t] = z[z>t]-t
    x0[z<-t] = z[z<-t]+t
    return x0

def calculate(lam,x_train,x_test,y_train,y_test):
    eig, _ = np.linalg.eig(transpose(x_train) @ x_train)
    max_eig = np.real(eig[0])
    miu = 1/max_eig
    beta = np.zeros((14, 1))
    err=1
    while(err > eps):
        z = beta - miu * (transpose(x_train)@ x_train @ beta - transpose(x_train) @ y_train)
        z=np.array(z)
        z = shrink(z, miu*lam)
        err = np.sum(np.square(beta - z))
        beta = z
    mse=np.sum(np.square(y_test - x_test @ beta))/shape(y_test)[0]
    re=[]
    re.append(mse)
    re.append(beta)
    return re

#####lamda网格搜索
out=[]
for i in range(0, 61):
    out1=calculate(10**(-3+0.1*i),x_train1,x_test1,y_train1,y_test1)
    out2=calculate(10**(-3+0.1*i),x_train2,x_test2,y_train2,y_test2)
    out3=calculate(10**(-3+0.1*i),x_train3,x_test3,y_train3,y_test3)
    out4=calculate(10**(-3+0.1*i),x_train4,x_test4,y_train4,y_test4)
    out5=calculate(10**(-3+0.1*i),x_train5,x_test5,y_train5,y_test5)
    mean_mse=(out1[0]+out2[0]+out3[0]+out4[0]+out5[0])/5
    out.append(mean_mse)
min_mse=1000
indx=0
for i in range(0, 61):
    if(out[i]<min_mse):
        min_mse=out[i]
        indx=i
        
#平均SSE最小时，lamda所加的0.1s,和虽小的平均残差平方和
print(indx, min_mse)
fina=calculate(10**(-3+0.1*indx),x_train1,x_test1,y_train1,y_test1)

#第一折的结果系数beta
print(fina[1])

17 84.09492423448225
[[-1.33098402e-01]
 [ 1.55741059e-01]
 [-1.14952196e-01]
 [ 1.47331405e-02]
 [ 7.50628914e-03]
 [ 2.46443085e-01]
 [ 1.46750872e-01]
 [ 4.86339664e-02]
 [ 5.01344128e-02]
 [ 1.37793844e-04]
 [ 1.89737739e-01]
 [ 3.71618878e-02]
 [-5.35565947e-01]
 [ 2.21306625e-02]]


精确度设为1e-9，从 10^−3+0.1*s, s = 0,...,60 中通过5 交叉验证，选择测试集上平均残差平方最小的λ。最终选取λ为10^−3+1.7=10^−1.3，此时五折交叉验证的平均残差平方和为84.095。取平均残差平方和最小的lamda,所对应的第一折训练得到的自变量的系数如上所示，其中最后一个为截距项。
可以看出城镇犯罪率、每个城镇非零售的业务比例、人口状况下降这三个自变量和因变量是负相关的，其他几个变量对与因变量是正相关的。变量每10000美元的全额物业税率的系数相对来说最小，约为0.0001，比较接近于0

In [8]:
##相对收敛标准，lamda网格搜索，五折交叉验证
eps2 = 1e-9

def shrink(z, t):
    assert(t>0)
    x0 = np.zeros(z.shape)
    x0[z>t] = z[z>t]-t
    x0[z<-t] = z[z<-t]+t
    return x0

def calculate2(lam,x_train,x_test,y_train,y_test):
    eig, _ = np.linalg.eig(transpose(x_train) @ x_train)
    max_eig = np.real(eig[0])
    miu = 1/max_eig
    beta = np.zeros((14, 1))+1
    err2=1
    while(err2 > eps2):
        z = beta - miu * (transpose(x_train)@ x_train @ beta - transpose(x_train) @ y_train)
        z=np.array(z)
        z = shrink(z, miu*lam)
        err2 = np.sum(np.square(beta - z))/np.sum(np.square(beta))
        beta = z
    mse=np.sum(np.square(y_test - x_test @ beta))/shape(y_test)[0]
    re=[]
    re.append(mse)
    re.append(beta)
    return re

out=[]
for i in range(0, 61):
    out1=calculate2(10**(-3+0.1*i),x_train1,x_test1,y_train1,y_test1)
    out2=calculate2(10**(-3+0.1*i),x_train2,x_test2,y_train2,y_test2)
    out3=calculate2(10**(-3+0.1*i),x_train3,x_test3,y_train3,y_test3)
    out4=calculate2(10**(-3+0.1*i),x_train4,x_test4,y_train4,y_test4)
    out5=calculate2(10**(-3+0.1*i),x_train5,x_test5,y_train5,y_test5)
    mean_mse=(out1[0]+out2[0]+out3[0]+out4[0]+out5[0])/5
    out.append(mean_mse)
min_mse=1000
indx=0
for i in range(0, 61):
    if(out[i]<min_mse):
        min_mse=out[i]
        indx=i
        
#平均SSE最小
print(indx, min_mse)
fina2=calculate2(10**(-3+0.1*indx),x_train1,x_test1,y_train1,y_test1)

#第一折的结果系数
print(fina2[1])

60 78.99840322851298
[[-0.03977975]
 [ 0.14184849]
 [ 0.52427849]
 [ 0.93649304]
 [ 0.92358325]
 [ 1.00396725]
 [ 0.04092692]
 [ 0.84760517]
 [ 0.74756845]
 [-0.05839717]
 [ 0.6380743 ]
 [ 0.0263526 ]
 [-0.20886355]
 [ 0.92431975]]


通过相对收敛标准，同一精确度标准下，比绝对收敛标准要求下运行时间更长，最终选取lambda为10^3，平均残差平方和约为78.9984，效果好于同一精度下的绝对收敛标准，二者同一变量的系数估计的正负性相同。